In [1]:
!pip install bs4
!pip install spacy
!python -m spacy download en_core_web_sm

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [62]:
import spacy
import re
import pandas as pd
import numpy as np
import bs4
import requests
from spacy import displacy

nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher 
from spacy.tokens import Span 

import networkx as nx

import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', 200)
%matplotlib inline


In [63]:
def read_file(filename):
    with open(filename, encoding='utf-8', mode='r', errors='ignore') as file:
        return file.read().strip() 


In [64]:
book = read_file('data/Strang-Linear Algebra.txt')

In [69]:
def get_one_chapter(chapter_number, book):
    s_chapter_number = str(chapter_number)
    chapter_starts = re.search('Chapter\s' + s_chapter_number +'\s\n.*\s\n' + s_chapter_number + '\.1.*\n', book)
    chapter_ends = re.search('Chapter\s' + str(chapter_number + 1) + '\s\n.*\s\n' + str(chapter_number + 1) + '\.1', 
                             book[chapter_starts.start():])
    chapter = book[chapter_starts.end():chapter_starts.end()+chapter_ends.start()]
      
    # TODO: check if this gavnocode works
    problem_set_start = re.search('Problem\sSet\s' + s_chapter_number + '\.[0-9]+\s', chapter)

    while problem_set_start:
        if chapter.count('Problem\sSet\s') == 1:  # very clever ifka, only sverhrazums top 5 understand
            chapter = chapter[:problem_set_start.start()]
            break
        problem_set_end = re.search(s_chapter_number + '\.[0-9]+\s.*\n', chapter[problem_set_start.end():])
        chapter = chapter[:problem_set_start.start()] + chapter[problem_set_start.end() + problem_set_end.start():]
        problem_set_start = re.search('Problem\sSet\s' + s_chapter_number + '\.[0-9]+\s', chapter)

        
    return re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', chapter.replace('\n', ' ').strip())


In [70]:
def sentence_fits(s):
    """Check if sentence contains only one subject and one object"""
    deps = ''.join([token.dep_ for token in nlp(s)])
    return deps.count('obj') == 1 & deps.count('subj') == 1
    

In [71]:
chapter = get_one_chapter(1, book)

sentences = [i for i in chapter if sentence_fits(i)]
sentences[:25]

['This  should work and it does: 4 times (x = .1) plus 5 times (y = 2) equals 6.  2.',
 'Elimination wins (and this is even the best way to  compute determinants).',
 'Singular cases have none or too many.',
 'The computer can do those quickly, but not many trillions.',
 'We want to see them both.',
 'The first approach concentrates on the separate equations (the rows).',
 'The second  equation x+y = 5 produces a second line (Figure 1.2a).',
 'It is the only solution to both equations.',
 'Geometrically this produces a famous parallelogram.',
 'The column picture confirms that x = 2 and y = 3.',
 'It contains the points (52 ;0;0) and (0;5;0) and (0;0;5).',
 'That  triple intersection point (1;1;2) solves the linear system.',
 'How does this row picture extend into n dimensions?',
 'The n equations will contain  n unknowns.',
 'The first equation still determines a plane.',
 'Finally a fourth plane leaves a single point.',
 'You can choose the arrow, or the point, or the three numbers.'

In [72]:
def get_relation(sent):
    doc = nlp(sent)

    # Matcher class object 
    matcher = Matcher(nlp.vocab)

    #define the pattern 
    pattern = [{'DEP':'ROOT'}, 
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},  
            {'POS':'ADJ','OP':"?"}] 

    matcher.add("matching_1", None, pattern) 

    matches = matcher(doc)
    k = len(matches) - 1

    span = doc[matches[k][1]:matches[k][2]] 

    return(span.text)


In [73]:
relations = [get_relation(i) for i in sentences]
pd.Series(relations).value_counts()[:25]

is               22
have             10
are               8
be                4
produces          3
Matrices          2
gives             2
came from         2
U.                2
was               2
20                2
13                2
is in             2
solves            2
Suppose           2
56                2
has               2
know              2
contains          2
happen at         1
are about         1
Note              1
permute           1
executes          1
be consistent     1
dtype: int64

In [74]:
def get_entities(sent):
    ## chunk 1
    ent1 = ""
    ent2 = ""

    prv_tok_dep = ""    # dependency tag of previous token in the sentence
    prv_tok_text = ""   # previous token in the sentence

    prefix = ""
    modifier = ""

    for tok in nlp(sent):
        ## chunk 2
        # if token is a punctuation mark then move on to the next token
        if tok.dep_ != "punct":
            # check: token is a compound word or not
            if tok.dep_ == "compound":
                prefix = tok.text
                # if the previous word was also a 'compound' then add the current word to it
                if prv_tok_dep == "compound":
                    prefix = prv_tok_text + " "+ tok.text

            # check: token is a modifier or not
            if tok.dep_.endswith("mod") == True:
                modifier = tok.text
                # if the previous word was also a 'compound' then add the current word to it
                if prv_tok_dep == "compound":
                    modifier = prv_tok_text + " "+ tok.text

            ## chunk 3
            if tok.dep_.find("subj") == True:
                ent1 = modifier +" "+ prefix + " "+ tok.text
                prefix = ""
                modifier = ""
                prv_tok_dep = ""
                prv_tok_text = ""      

            ## chunk 4
            if tok.dep_.find("obj") == True:
                ent2 = modifier +" "+ prefix +" "+ tok.text

            ## chunk 5  
            # update variables
            prv_tok_dep = tok.dep_
            prv_tok_text = tok.text

    return [ent1.strip(), ent2.strip()]


In [76]:
objects = np.empty(len(sentences), dtype=)
subjects = np.empty(len(sentences))

for i, s in enumerate(sentences):
    objects[i], subjects[i] = get_entities(s)
    
subjects[:25]

ValueError: could not convert string to float: '4  times'

In [16]:
subjects = [i[0] for i in entities]
objects = [i[1] for i in entities]

kg_df = pd.DataFrame({'source':source, 'target':target, 'edge':relations})


[['', ''],
 ['', ''],
 ['We', ''],
 ['I', ''],
 ['', ''],
 ['', ''],
 ['', ''],
 ['', ''],
 ['', ''],
 ['This', ''],
 ['Determinants', ''],
 ['', ''],
 ['They', ''],
 ['', ''],
 ['', ''],
 ['It', ''],
 ['We', ''],
 ['', ''],
 ['This', ''],
 ['', ''],
 ['Equations', ''],
 ['', ''],
 ['', ''],
 ['', ''],
 ['', ''],
 ['It', ''],
 ['', ''],
 ['They', ''],
 ['', ''],
 ['It', ''],
 ['It', ''],
 ['', ''],
 ['', ''],
 ['We', ''],
 ['We', ''],
 ['', ''],
 ['I', ''],
 ['', ''],
 ['', ''],
 ['', ''],
 ['', ''],
 ['', ''],
 ['', ''],
 ['', ''],
 ['', ''],
 ['', ''],
 ['', ''],
 ['We', ''],
 ['', ''],
 ['', '']]

In [44]:
get_entities('the film had 200 patents')

['', '']